# Monte Carlo -- 3 Implementations 
# Pure Python, Numpy, Fully Vectorized Numpy

Black Scholes: 
    $dS_t = r S_t dt + \sigma S_t dZ_t$
Difference Equation:
    $S_t = S_(t - \Delta t) exp((r - 0.5 \sigma^2)\Delta t + \sigma \sqrt(\Delta t) Z_t$

In [1]:
import math
from scipy import stats
import pandas as pd
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
from time import time
import numpy as np
from math import exp, sqrt, log
from random import gauss, seed
%matplotlib inline

In [2]:

def bsm_call_value(S_0, K, T, r, sigma):
    """European Call option model via BSM.
    Nothing exotic or American about this, simple
    bsm w/o optimal execution time etc.
    
    Parameters
    ----------
    S_0: float
        initial stock price
    K: float
        strike price
    T: float
        time to maturity
    r: float
        interest rate
    sigma: float
        implied volatility factor in diffusion term form
    
    Returns
    -----------
    value: float
        calculated value of European Call option
    
    """
    
    s0 = float(S_0)
    d1 = (math.log(s0/K) + (r + 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T)) # upper bound
    d2 = (math.log(s0/K) + (r - 0.5 * sigma ** 2) * T) / (sigma * math.sqrt(T)) # lower bound
    return (s0 * stats.norm.cdf(d1, 0., 1.0) - K * math.exp(-r * T) * stats.norm.cdf(d2, 0., 1.0))

In [3]:
print("The actual value of the call option", 
      bsm_call_value(100, 105, 1, 0.05, 0.2))

The actual value of the call option 8.02135223514


# Exact option Value from previous notebook

# Recipe:

* Divide time interval from [0, T]
* For i in __I__
    * Determine the time T value of the index level $S_T(i)$ by applying the pseudo-random number time step by time step to the discretization scheme in SDE eqn above
    * Determine $max(S_t(i) - K, 0)$ that is inner value of $h_T(S_T(i))$

# Pure Python Implementation

In [4]:
seed(10)

S0 = 100
K = 105
T = 1.0
r = 0.05
sigma = 0.2
M = 50 # number of steps to run the algo
dt = T / M #step size
I = 250000 # Number of potential paths

S = []
# Calculate the price for each path and for each step
for i in range(I):
    path = []
    for t in range(M + 1):
        if t == 0:
            path.append(0)
        else:
            z = gauss(0.0, 1.0)
            St = path[t-1] * exp((r - 0.5 * sigma ** 2) *dt 
                                + sigma * sqrt(dt) * z)
            path.append(St)
        S.append(path)
        


In [5]:
C0 = exp(-r * T) #* sum([max(path[-1] - K, 0) for path in S]) / I
print(C0)

0.951229424500714
